In [2]:
import numpy as np
import pandas as pd
import paramiko
import re
import copy

###### Read data from server

In [2]:
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(hostname='130.60.75.213', port=22, username='baml_02', password='CHANGE_6b4ba0bc')
stdin, stdout, stderr = ssh.exec_command('ls')
sftp_client = ssh.open_sftp()
remote_file = sftp_client.open('train.tueba.coref.txt')
sftp_client2 = ssh.open_sftp()
remote_file2 = sftp_client2.open('train.tueba.sync.txt')

/home/qiaowen_wang97/anaconda3/lib/python3.5/site-packages/cryptography/hazmat/backends/openssl/ciphers.py:114: UserWarning: implicit cast from 'char *' to a different pointer type: will be forbidden in the future (check that the types are as you expect; use an explicit ffi.cast() if they are correct)
  operation
/home/qiaowen_wang97/anaconda3/lib/python3.5/site-packages/cryptography/hazmat/backends/openssl/ciphers.py:140: UserWarning: implicit cast from 'char *' to a different pointer type: will be forbidden in the future (check that the types are as you expect; use an explicit ffi.cast() if they are correct)
  self._backend._ffi.from_buffer(data), len(data)


In [3]:
sync = []
for line in remote_file2: 
    if "#begin document" not in line:
        if "#end document" not in line:
#             line = line[:-2]
            line = line.strip('\n')
            line = line.split('\t')
#             print(line)
            sync.append(line)
            continue 

/home/qiaowen_wang97/anaconda3/lib/python3.5/site-packages/cryptography/hazmat/backends/openssl/ciphers.py:140: UserWarning: implicit cast from 'char *' to a different pointer type: will be forbidden in the future (check that the types are as you expect; use an explicit ffi.cast() if they are correct)
  self._backend._ffi.from_buffer(data), len(data)


In [4]:
#rename columns and drop unused colunms
df_sync = pd.DataFrame(sync, columns =['doc_id', 'word_number','?', 'word','lemma','??','POS','FEATS','head','gramFct','????','?????'])
df_sync2 = df_sync.drop(columns=['?','????','?????'])
df_sync3 = df_sync2[~df_sync2['doc_id'].isin([''])]
df_sync4 = df_sync3.reset_index(drop = True)

In [6]:
new_df2 = []
for line in remote_file: 
    if "#begin document" not in line:
        if "#end document" not in line:
            line = line.strip('\n')
            line = line.split('\t')
            new_df2.append(line)
            continue 

/home/qiaowen_wang97/anaconda3/lib/python3.5/site-packages/cryptography/hazmat/backends/openssl/ciphers.py:140: UserWarning: implicit cast from 'char *' to a different pointer type: will be forbidden in the future (check that the types are as you expect; use an explicit ffi.cast() if they are correct)
  self._backend._ffi.from_buffer(data), len(data)


In [7]:
#rename columns and drop unused colunms
df2 = pd.DataFrame(new_df2, columns = ['doc_id', 'sentence', 'word_number', 'word','tagging','?','lemma','first-','second-','third-','fourth*','fifth-','coreference_chain_id' ])
df3 = df2.drop(columns=['lemma','first-','second-','third-','fourth*','fifth-'])
df4=df3[~df3['doc_id'].isin([''])]
df4 = df4.reset_index(drop=True)

In [ ]:
#Calculate how many words in one sentences in each segment in train_coref file
df5 = df4.groupby(['doc_id','sentence']).count().reset_index()
df5.drop(columns=['word','tagging','?','coreference_chain_id'],inplace=True)

###### Class for Stack and Node

In [ ]:
class Node:  
    def __init__(self, value):
        self.value = value
        self.next = None
  
  
class Stack: 
    def __init__(self):
        self.top = None
  
    def push(self, value):
        node = Node(value)
        node.next = self.top
        self.top = node
  
    def pop(self):
        node = self.top
        if node is None:
            raise Exception('This is an empty stack')
        self.top = node.next
        return node.value
  
    def peek(self):
        node = self.top
        if node is None:
            raise Exception('This is an empty stack')
        return node.value
  
    def is_empty(self):
        return not self.top
  
    def size(self):
        node = self.top
        count = 0
        if node is None:
            raise Exception('This is an empty stack')
        while node is not None:
            count += 1
            node = node.next
        return count

###### Extract gold chain from train_coref file
First : extract mentions

In [11]:
stack = Stack()
stack_temp = Stack()
data_len = len(df4)
for i in range(0, data_len):
#     print(i)
    sentid = df4.iloc[i]['sentence']
    tokenid = df4.iloc[i]['word_number']
    seg = df4.iloc[i]['doc_id']
#     if df3.iloc[i]['coreference_chain_id'] != '-':
#     x=type(df3.iloc[i]['coreference_chain_id'])
#      print(x)
    if df4.iloc[i]['coreference_chain_id'].find('|'):
        ids = df4.iloc[i]['coreference_chain_id'].split('|')
    else:
        ids = df4.iloc[i]['coreference_chain_id']
    for identry in ids:
#         print(identry)
        co_id_list = re.findall(r'\d+', identry)
        if len(co_id_list):
            co_id = co_id_list[0]
        if identry.startswith('(') and identry.endswith(')'):
            stack.push([co_id,seg,sentid,tokenid,tokenid])
        elif identry.startswith('('):
                stack_temp.push([co_id,seg,sentid,tokenid])
        elif identry.endswith(')'):
                lastentry = stack_temp.pop()
                lastentry.append(tokenid)
                stack.push(lastentry)

In [12]:
#copy stack generated last step for further use
stack_test = Stack()
stack_test = copy.copy(stack)
stack_test_2 = copy.copy(stack)
stack_test_3 = copy.copy(stack)

In [14]:
#Pop mentions from the stack and store them in a dataframe
chain = pd.DataFrame()
for i in range(0, stack.size()):
    top  = stack_test.pop()
    chain = chain.append([top])

In [15]:
#reset index and rename columns
chain.reset_index(drop=True)
chain.columns = ['core_id', 'doc_id', 'sent_num', 'phrase_start','phrase_end']
chain = chain.reset_index(drop=True)

In [16]:
#copy data generated last step for further use
chain_test = copy.copy(chain)

In [17]:
group = chain_test.groupby('doc_id').size()

In [18]:
#Calculate how many mentions in one segment
groupby = chain_test.groupby('doc_id').size()

In [19]:
groupby.reset_index()

,doc_id,0
0,T890102.141,135
1,T890102.154,39
2,T890102.38,89
3,T890102.50,250
4,T890102.82,74
5,T890102.84,148
6,T890103.208,69
7,T890103.246,94
8,T890104.37,59
9,T890104.86,63


In [20]:
#rename columns
groupby.columns = ['doc_id','total_counts']

In [21]:
#copy again for further use
df_groupby = groupby.to_frame()
df_groupby = df_groupby.rename(columns = {'doc_id':'doc_id',0:'total_counts'})
df_groupby = df_groupby.reset_index()

In [22]:
#Calculate how many mentions in one segment again
df_groupby_2 = chain_test.groupby('doc_id').count().reset_index()

In [23]:
#Calculate how many mentions in one segment with the same coref_id
groupby_2 = chain_test.groupby(['core_id','doc_id']).count().reset_index()

In [24]:
#Merge the dataframe for all mentions with the count for all coref_id(the dataframe generate last step)
result = chain_test.merge(groupby_2,on=['core_id','doc_id'],how='outer')
result.drop(columns=['phrase_end_y'],inplace=True)
result.drop(columns=['sent_num_y'],inplace=True)
result = result.rename(columns={'sent_num_x':'sent_num','phrase_start_x':'phrase_start','phrase_end_x':'phrase_end','phrase_start_y':'counts'})

In [ ]:
#Merge the dataframe from last step and the dataframe with count for mentions in each segment information in it.
#The new dataframe contains informations about count for a single coref_id and count for all mentions in one segment.
result2 = result.merge(df_groupby,on=['doc_id'],how='outer')
result2 = result2.rename(columns={0:'total_counts'})

Second: generate goldchian use the dataframe generate last step and the algorithm from lecture 5

goldchain

In [26]:
gold_chain_dict = dict() 
i = 0
while i < len(result2):
    k = 0
    gold_chain_dict[result2.iloc[i]['doc_id']] = [[(result2.iloc[i]['sent_num'],result2.iloc[i]['phrase_start'],result2.iloc[i]['phrase_end'])]]
    j=0
    for j in range(result2.iloc[i]['total_counts']):
        if i+1 < len(result2):
            #if 2 mentions are in the same segment
            if result2.iloc[i+1]['doc_id'] == result2.iloc[i]['doc_id']:
                #if 2 mentions have same coref_id
                if result2.iloc[i+1]['core_id'] == result2.iloc[i]['core_id']:
                    gold_chain_dict[result2.iloc[i]['doc_id']][k].append((result2.iloc[i+1]['sent_num'],result2.iloc[i+1]['phrase_start'],result2.iloc[i+1]['phrase_end']))
                #if they do not have same coref_id then create a new list for the new coref_id
                elif result2.iloc[i+1]['core_id'] != result2.iloc[i]['core_id']:
                    k = k+1
                    gold_chain_dict[result2.iloc[i]['doc_id']].append([(result2.iloc[i+1]['sent_num'],result2.iloc[i+1]['phrase_start'],result2.iloc[i+1]['phrase_end'])])
            i = i+1
    if i+1 >= len(result2):
         break

goldchain with coref_if info

In [27]:
gold_chain_dict_with_core_id = dict() 
i = 0
while i < len(result2):
    k = 0
    gold_chain_dict_with_core_id[result2.iloc[i]['doc_id']] = [[(result2.iloc[i]['core_id'],result2.iloc[i]['sent_num'],result2.iloc[i]['phrase_start'],result2.iloc[i]['phrase_end'])]]
    j=0
    for j in range(result2.iloc[i]['total_counts']):
        if i+1 < len(result2):
            #if 2 mentions are in the same segment
            if result2.iloc[i+1]['doc_id'] == result2.iloc[i]['doc_id']:
                #if 2 mentions have same coref_id
                if result2.iloc[i+1]['core_id'] == result2.iloc[i]['core_id']:
                    gold_chain_dict_with_core_id[result2.iloc[i]['doc_id']][k].append((result2.iloc[i+1]['core_id'],result2.iloc[i+1]['sent_num'],result2.iloc[i+1]['phrase_start'],result2.iloc[i+1]['phrase_end']))
                #if they do not have same coref_id then create a new list for the new coref_id
                elif result2.iloc[i+1]['core_id'] != result2.iloc[i]['core_id']:
                    k = k+1
                    gold_chain_dict_with_core_id[result2.iloc[i]['doc_id']].append([(result2.iloc[i+1]['core_id'],result2.iloc[i+1]['sent_num'],result2.iloc[i+1]['phrase_start'],result2.iloc[i+1]['phrase_end'])])
            i = i+1
    if i+1 >= len(result2):
         break

In [28]:
group = groupby.reset_index()
group.columns = ['doc_id','counts']

In [29]:
#copy for further use
gold_chain_dict_dup = dict()
gold_chain_dict_dup = copy.deepcopy(gold_chain_dict)

In [30]:
#Reprocess the train_coref data for further use
df7 = df2.drop(columns=['first-','second-','third-','fourth*','fifth-'])
df8 = df7[~df7['doc_id'].isin([''])]
df8.reset_index(drop=True)
df9 = df8.reset_index(drop = True)

In [31]:
#store segment_id info in a list
df_groupby_2_to_list = df_groupby_2['doc_id'].tolist()

In [32]:
#change data type for further use
df5['word_number'] = df5['word_number'].astype('int')

In [ ]:
#save files
json_str = json.dumps(gold_chain_dict_with_core_id,indent = 4)
with open('gold_chain_dict_with_core_id.json','w')as json_file:
    json_file.write(json_str)
df9.to_csv(path_or_buf='df9.csv',sep = ','index = True, header = True,encoding='UTF-8')
df_groupby_2.to_csv(path_or_buf='df_groupby_2.csv',sep = ','index = True, header = True,encoding='UTF-8')
df_sync4.to_csv(path_or_buf='df_sync4.csv',sep = ','index = True, header = True,encoding='UTF-8')
df5.to_csv(path_or_buf='df5.csv',sep = ','index = True, header = True,encoding='UTF-8')

###### If already have files named "df9.csv", "df_groupby_2.csv","df_sync4.csv" ,"df5.csv"and "gold_chain_dict_with_core_id.json" then please run the program from here.

In [3]:
#read files
df9 = pd.read_csv("df9.csv",sep = ',').drop(columns=['Unnamed: 0'])
df_groupby_2 = pd.read_csv("df_groupby_2.csv",sep = ',').drop(columns=['Unnamed: 0'])
df_sync4 = pd.read_csv("df_sync4.csv",sep = ',').drop(columns=['Unnamed: 0'])
df5 = pd.read_csv("df5.csv",sep = ',').drop(columns=['Unnamed: 0'])

In [4]:
#read files
import json
with open('gold_chain_dict_with_core_id.json','r') as f:
    gold_chain_dict_with_core_id = json.load(fp=f)
    print(gold_chain_dict_with_core_id)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [10]:
gold_chain_dict_with_core_id

{'T990430.79': [[['0', '18176', '15', '16'],
   ['0', '18175', '10', '11'],
   ['0', '18173', '1', '1'],
   ['0', '18172', '1', '18'],
   ['0', '18171', '14', '15'],
   ['0', '18170', '4', '4']],
  [['1', '18176', '5', '7'],
   ['1', '18173', '8', '8'],
   ['1', '18171', '20', '21']],
  [['2', '18175', '6', '6'], ['2', '18171', '1', '1']]],
 'T990507.152': [[['1', '1869', '3', '5'], ['1', '1867', '4', '5']],
  [['0', '1868', '8', '8'], ['0', '1868', '1', '3'], ['0', '1867', '1', '2']]],
 'T970307.42': [[['12', '57020', '2', '3'],
   ['12', '57018', '9', '10'],
   ['12', '57007', '22', '22'],
   ['12', '57007', '19', '20']],
  [['3', '57019', '1', '1'],
   ['3', '57014', '22', '23'],
   ['3', '57014', '1', '1'],
   ['3', '57007', '47', '47'],
   ['3', '57006', '34', '34'],
   ['3', '57002', '21', '22'],
   ['3', '57000', '5', '5'],
   ['3', '56999', '1', '6'],
   ['3', '56992', '7', '10']],
  [['10', '57018', '1', '4'],
   ['10', '57006', '2', '2'],
   ['10', '57005', '28', '32']],
  [[

###### Second: use the chain: "gold_chain_dict_with_core_id"  to generate a new dataframe with mentions and features in it

In [ ]:
golden_list = list()
golden_dict_test = dict()
i=0
for seg in df_groupby_2['doc_id']:
    #find all coreference chains in one segment
    segchain = gold_chain_dict_with_core_id.get(seg)
    w=0
    #for each chain in chains
    for chain in segchain:
        for j in range(len(chain)):
            doc_id = seg
            core_id = chain[j][0] 
            sent_id = chain[j][1]
            start_token = chain[j][2]
            end_token = chain[j][3]
            #find record in train_coref file to extract features
            index = df9[(df9['doc_id'] == doc_id) & (df9['sentence'] == int(sent_id)) & (df9['word_number'] == int(start_token))].index.tolist()
            POS = ""
            gramFct = ""
            FEATS = ""
            head_num = float("inf")
            #find the head for a mention with multiple words in it.
            if int(start_token) != int(end_token) :
                dis = int(end_token) - int(start_token)+1
                for k in range(dis):
                    head = int(df_sync4.iloc[index[0]+k]['head'])
                    if head <= head_num:
                        head_num = head
                        POS = df9.iloc[index[0]+k]['tagging']
                        gramFct = df_sync4.iloc[index[0]+k]['gramFct']
                        FEATS = df_sync4.iloc[index[0]+k]['FEATS']
            #if there is only one word in the mention.
            else: 
                POS = df9.iloc[index[0]]['tagging']
                head_num = int(df_sync4.iloc[index[0]]['head'])
                gramFct = df_sync4.iloc[index[0]]['gramFct']
                FEATS = df_sync4.iloc[index[0]]['FEATS']
            golden_list.append([core_id,seg,sent_id,start_token,end_token,gramFct,POS,head_num,FEATS])
            i=i+1

In [12]:
#store the list in a dataframe and rename the columns
df_golden_list = pd.DataFrame(golden_list, columns=['core_id','seg','sent_id','start_token','end_token','gramFct','POS','head','FEATS']) 

In [14]:
#save file
df_golden_list.to_csv(path_or_buf='df_golden_list', sep=',',index = True, header = True,encoding='UTF-8')

In [6]:
#read file
df_golden_list=pd.read_csv('df_golden_list.csv',sep=',')
df_golden_list = df_golden_list.drop(columns=['Unnamed: 0'])
df_golden_list_2 = df_golden_list

In [9]:
#change the data types for further use
df_golden_list_2['sent_id'] = df_golden_list_2['sent_id'].astype('int')
df_golden_list_2['start_token'] = df_golden_list_2['start_token'].astype('int')
df_golden_list_2['start_token'] = df_golden_list_2['start_token'].astype('int')
df_golden_list_2['start_token'] = df_golden_list_2['start_token'].astype('int')

In [13]:
#sort by 'sent_id' and 'start_token'
df_golden_list_sort = df_golden_list_2.sort_values(by=['sent_id','start_token'],ascending=[False,False])

In [14]:
df_golden_list_sort = df_golden_list_sort.reset_index(drop=True)

In [17]:
df_golden_list_sort

,core_id,seg,sent_id,start_token,end_token,gramFct,POS,head,FEATS
0,0,T890221.92,67713,6,6,nmod:poss,PPOSAT,8,Case=Dat|Gender=Masc|Number=Sing|Poss=Yes
1,0,T890221.92,67713,3,3,nsubj:pass,PPER,15,Case=Nom|Gender=Fem|Number=Sing|PronType=Prs
2,33,T890221.92,67712,15,15,obl,NE,7,Case=Dat|Gender=Neut|Number=Sing
3,0,T890221.92,67712,9,9,obj,PPER,7,Case=Acc|Gender=Fem|Number=Sing|PronType=Prs
4,1,T890221.92,67711,16,16,obl,NE,17,Case=Dat|Gender=Neut|Number=Sing
5,0,T890221.92,67711,13,13,obj,PPER,17,Case=Acc|Gender=Fem|Number=Sing|PronType=Prs
6,0,T890221.92,67711,10,10,nmod:poss,PPOSAT,11,Case=Gen|Gender=Masc|Number=Plur|Poss=Yes
7,9,T890221.92,67711,4,5,det,ART,5,Case=Acc|Definite=Def|Gender=Masc|Number=Plur|...
8,0,T890221.92,67711,1,1,nsubj:pass,PPER,6,Case=Nom|Gender=Fem|Number=Sing|PronType=Prs
9,0,T890221.92,67710,2,2,nmod:poss,PPOSAT,3,Case=Dat|Gender=Fem|Number=Sing|Poss=Yes


###### Third: generate positive samples and negative samples for train 

Using the algorithm in lecture 5

In [ ]:
import operator
from collections import Counter
i = 0
gold_std = list()
gold_std_neg = list()
for seg in df_groupby_2['doc_id']:
    #find all coreference chains in one segment
    segchain = gold_chain_dict_with_core_id.get(seg)
    #for each chain in chains
    for chain in segchain:
            for j in range(len(chain)):
                if (j+1)<len(chain): 
                    #for each neighbor mentions
                    sentence1 = chain[j][1] 
                    sentence2 = chain[j+1][1]
                    core_id1 = chain[j][0]
                    core_id2 = chain[j+1][0]
                    start1 = chain[j][2]
                    start2 = chain[j+1][2]
                    end1 = chain[j][3]
                    end2 = chain[j+1][3]
                    #find records in the dataframe df_golden_list which contains informations about features
                    x1 = df_golden_list[(df_golden_list['sent_id']==int(sentence1)) & (df_golden_list['core_id']==int(core_id1)) & (df_golden_list['start_token']==int(start1)) &(df_golden_list['end_token']==int(end1))]
                    x2 = df_golden_list[(df_golden_list['sent_id']==int(sentence2)) & (df_golden_list['core_id']==int(core_id2)) & (df_golden_list['start_token']==int(start2)) &(df_golden_list['end_token']==int(end2))]
                    #extract features
                    POS1 = x1['POS'].tolist()[0]
                    POS2 = x2['POS'].tolist()[0]
                    if POS1 == POS2:
                        samePOS = 1
                    else : samePOS = 0
                    head1 = int(x1['head'])
                    head2 = int(x2['head'])
                    if head1 == head2:
                        sameHead = 1
                    else : sameHead = 0
                    gramFct1 = x1['gramFct'].tolist()[0]
                    gramFct2 = x2['gramFct'].tolist()[0]
                    if gramFct1 == gramFct2:
                        sameGramFct = 1
                    else : sameGramFct = 0
                    FEATS1 = x1['FEATS'].tolist()[0]
                    FEATS2 = x2['FEATS'].tolist()[0]
                    if FEATS1 == FEATS2:
                        matchOrNot = 1
                    else : matchOrNot = 0
                    #extract Case, Gender and Number info from FEATS
                    if 'Case' in FEATS1.split('|')[0]:
                        Case1 = FEATS1.split('|')[0].split('=')[1] 
                    else: Case1=0
                    if 'Case' in FEATS2.split('|')[0]:
                        Case2 = FEATS2.split('|')[0].split('=')[1] 
                    else: Case2=0
                    if Case1 == Case2:
                            sameCase = 1
                    else : sameCase = 0
                    for feat in FEATS1.split('|'):
                        if 'Gender' in feat:
                            Gender1 = feat.split('=')[1]
                            break
                        else: Gender1=0
                    for feat in FEATS2.split('|'):
                        if 'Gender' in feat:
                            Gender2 = feat.split('=')[1]
                            break
                        else: Gender2=0
                    if Gender1 == Gender2:
                            sameGender = 1
                    else : sameGender = 0     
                    for feat in FEATS1.split('|'):
                        if 'Number' in feat:
                            Number1 = feat.split('=')[1]
                            break
                        else: Number1=0
                    for feat in FEATS2.split('|'):
                        if 'Number' in feat:
                            Number2 = feat.split('=')[1]
                            break
                        else: Number2=0
                    if Number1 == Number2:
                            sameNumber = 1
                    else : sameNumber = 0     
                    label = 1
                    
                    #calculate distance
                    s_distance = 0
                    w_place_1 = 0
                    w_place_2 = 0
                    w_distance = 0
                    distance = int(chain[j][1])-int(chain[j+1][1])
                    
                    #if these two markables are not in the same or neighbor sentences
                    if distance >1:
                        #add all words between the two sentences where these two mentions' head exist
                        for k in range(1,distance):
                            if df5.loc[df5['sentence']==(int(chain[j+1][1])+k)]['word_number'].empty:
                                s_distance = s_distance+0
                            else:
                                s_distance = s_distance + int(df5.loc[df5['sentence']==(int(chain[j+1][1])+k)]['word_number'].tolist()[0])
                        #add the words before the second mention's head and behind the first mention's head
                        w_place_1 = int(df5.loc[df5['sentence']==int(chain[j+1][1])]['word_number'])-int(df_golden_list.loc[(df_golden_list['start_token']==int(chain[j+1][2])) & (df_golden_list['sent_id']==int(chain[j+1][1]))]['head'].tolist()[0])
                        w_place_2 = int(df_golden_list.loc[(df_golden_list['start_token']==int(chain[j][2])) & (df_golden_list['sent_id']==int(chain[j][1]))]['head'].tolist()[0])-1
                        w_distance = w_distance + s_distance + w_place_1 + w_place_2
                        
                    #if these two mentions are in neighbor sentences    
                    elif distance ==1:
                        #add the words before the second mention's head and behind the first mention's head
                        w_place_1 = int(df5.loc[df5['sentence']==int(chain[j+1][1])]['word_number'])-int(df_golden_list.loc[(df_golden_list['start_token']==int(chain[j+1][2])) & (df_golden_list['sent_id']==int(chain[j+1][1]))]['head'].tolist()[0])
                        w_place_2 = int(df_golden_list.loc[(df_golden_list['start_token']==int(chain[j][2])) & (df_golden_list['sent_id']==int(chain[j][1]))]['head'].tolist()[0])-1
                        w_distance = w_distance + s_distance + w_place_1 + w_place_2
                        
                    #if these two mentions are in same sentences
                    elif distance ==0:
                        #add words between heads of two mentions
                        w_place_1 = int(df_golden_list.loc[(df_golden_list['start_token']==int(chain[j+1][2])) & (df_golden_list['sent_id']==int(chain[j+1][1]))]['head'].tolist()[0])
                        w_place_2 = int(df_golden_list.loc[(df_golden_list['start_token']==int(chain[j][2])) & (df_golden_list['sent_id']==int(chain[j][1]))]['head'].tolist()[0])
                        if w_place_1 == w_place_2:
                            w_distance = w_distance + s_distance + abs(w_place_1 - w_place_2)
                        else : w_distance = w_distance + s_distance + abs(w_place_1 - w_place_2)-1
                            
                    #append the record to positive list
                    gold_std.append([label,w_distance,POS1,gramFct1,Case1,Gender1,Number1,POS2,gramFct2,Case2,Gender2,Number2,sameHead,samePOS,sameGramFct,matchOrNot,sameCase,sameGender,sameNumber]) 
                    
                    i= i+1
                    #find the record in sorted dataframe
                    index1 = int(df_golden_list_sort[(df_golden_list_sort['sent_id']==int(sentence1)) & (df_golden_list_sort['core_id']==int(core_id1)) & (df_golden_list_sort['start_token']==int(start1)) &(df_golden_list_sort['end_token']==int(end1))].index.tolist()[0])
                    index2 = int(df_golden_list_sort[(df_golden_list_sort['sent_id']==int(sentence2)) & (df_golden_list_sort['core_id']==int(core_id2)) & (df_golden_list_sort['start_token']==int(start2)) &(df_golden_list_sort['end_token']==int(end2))].index.tolist()[0])
                    
                    #for any mention exists between the indics of these two mentions
                    for w in range(index1,index2):
                        #one mention
                        n1 = df_golden_list_sort.loc[w]
                        #the next mention
                        n2 = df_golden_list_sort.loc[w+1]
                        #determine if these two mentions are in a segment
                        if Counter(n1['seg']) == Counter(n2['seg']):
                            #if these two mentions do not have the same coref_id
                            #then these two mentions can form a negative pair
                            if int(n1['core_id']) != int(n2['core_id']):
                                #some features
                                POS_n1 = n1['POS']
                                POS_n2 = n2['POS']
                                if POS_n1 == POS_n2:
                                     samePOS_n = 1
                                else : samePOS_n = 0
                                head_n1 = int(n1['head'])
                                head_n2 = int(n2['head'])
                                if head_n1 == head_n2:
                                    sameHead_n = 1
                                else : sameHead_n = 0
                                gramFct_n1 = n1['gramFct']
                                gramFct_n2 = n2['gramFct']
                                if gramFct_n1 == gramFct_n2:
                                    sameGramFct_n = 1
                                else : sameGramFct_n = 0
                                FEATS_n1 = n1['FEATS']
                                FEATS_n2 = n2['FEATS']
                                if FEATS_n1 == FEATS_n2:
                                    matchOrNot_n = 1
                                else : matchOrNot_n = 0
                                #extract Case, Gender and Number info from FEATS    
                                if 'Case' in FEATS_n1.split('|')[0]:
                                    Case_n1 = FEATS_n1.split('|')[0].split('=')[1] 
                                else: Case_n1=0
                                if 'Case' in FEATS_n2.split('|')[0]:
                                    Case_n2 = FEATS_n2.split('|')[0].split('=')[1] 
                                else: Case_n2=0
                                if Case_n1 == Case_n2:
                                     sameCase_n = 1
                                else : sameCase_n = 0
                                for feat in FEATS_n1.split('|'):
                                    if 'Gender' in feat:
                                        Gender_n1 = feat.split('=')[1]
                                        break
                                    else: Gender_n1=0
                                for feat in FEATS_n2.split('|'):
                                    if 'Gender' in feat:
                                        Gender_n2 = feat.split('=')[1]
                                        break
                                    else: Gender_n2=0
                                if Gender_n1 == Gender_n2:
                                    sameGender_n = 1
                                else : sameGender_n= 0  
                                
                                for feat in FEATS_n1.split('|'):
                                    if 'Number' in feat:
                                        Number_n1 = feat.split('=')[1]
                                        break
                                    else: Number_n1=0
                                for feat in FEATS_n2.split('|'):
                                    if 'Number' in feat:
                                        Number_n2 = feat.split('=')[1]
                                        break
                                    else: Number_n2=0
                                if Number_n1 == Number_n2:
                                    sameNumber_n = 1
                                else : sameNumber_n = 0   
                                label_n = 0
                                
                                #calculate distance
                                n_s_distance = 0
                                n_w_place_1 = 0
                                n_w_place_2 = 0
                                n_w_distance = 0
                                n_distance = abs(int(n1['sent_id'])-int(n2['sent_id']))
                                
                                #if these two markables are not in the same or neighbor sentences
                                if n_distance >1:
                                    #add all words between the two sentences where these two mentions' head exist
                                    for k in range(1,n_distance):
                                        if df5.loc[df5['sentence']==(df_golden_list_sort.loc[w+1]['sent_id']+k)]['word_number'].empty: 
                                            n_s_distance = n_s_distance+0
                                        else:
                                            n_s_distance = n_s_distance + int(df5.loc[df5['sentence']==(df_golden_list_sort.loc[w+1]['sent_id']+k)]['word_number'].tolist()[0])
                                    #add the words before the second mention's head and behind the first mention's head
                                    n_w_place_1 = int(df5.loc[df5['sentence']==df_golden_list_sort.loc[w+1]['sent_id']]['word_number'])-int(df_golden_list_sort.loc[w+1]['head'])
                                    n_w_place_2 = int(df_golden_list_sort.loc[w]['head'])-1
                                    n_w_distance = n_w_distance + n_s_distance + n_w_place_1 + n_w_place_2
                                    
                                #if these two mentions are in neighbor sentences    
                                elif n_distance ==1:
                                     #add words between heads of two mentions
                                    n_w_place_1 = int(df5.loc[df5['sentence']==df_golden_list_sort.loc[w+1]['sent_id']]['word_number'])-int(df_golden_list_sort.loc[w+1]['head'])
                                    n_w_place_2 = int(df_golden_list_sort.loc[w]['head'])-1 
                                    n_w_distance = n_w_distance + n_s_distance + n_w_place_1 + n_w_place_2
                                    
                                #if these two mentions are in same sentences
                                elif n_distance ==0:
                                    #add words between heads of two mentions
                                    n_w_place_1 = int(df_golden_list_sort.loc[w+1]['head'])
                                    n_w_place_2 = int(df_golden_list_sort.loc[w]['head'])
                                    if n_w_place_1 == n_w_place_2:
                                        n_w_distance = n_w_distance + n_s_distance + abs(n_w_place_1 - n_w_place_2)
                                    else : n_w_distance = n_w_distance + n_s_distance + abs(n_w_place_1 - n_w_place_2)-1
                                        
                                #append the record to negative list
                                gold_std_neg.append([label_n,n_w_distance,POS_n1,gramFct_n1,Case_n1,Gender_n1,Number_n1,POS_n2,gramFct_n2,Case_n2,Gender_n2,Number_n2,sameHead_n,samePOS_n,sameGramFct_n,matchOrNot_n,sameCase_n,sameGender_n,sameNumber_n])
    
                else:break

In [22]:
#convert list to dataframe for both negative list and positive list
df_gold_std = pd.DataFrame(gold_std, columns=['label','distance','POS1','gramFct1','Case1','Gender1','Number1','POS2','gramFct2','Case2','Gender2','Number2','sameHead','samePOS','sameGramFct','matchOrNot','sameCase','sameGender','sameNumber'])
df_gold_std_neg = pd.DataFrame(gold_std_neg, columns=['label','distance','POS1','gramFct1','Case1','Gender1','Number1','POS2','gramFct2','Case2','Gender2','Number2','sameHead','samePOS','sameGramFct','matchOrNot','sameCase','sameGender','sameNumber'])

In [25]:
#append negative samples and positive samples together form a new dataframe which contains all samples
df_gold_std_all = df_gold_std.append(df_gold_std_neg)

In [26]:
#save train file
df_gold_std_all.to_csv(path_or_buf='df_gold_std_all_18_12_with_case_and_gender_number.csv', sep=',',index = True, header = True,encoding='UTF-8')